# Age, Gender, Race Prediction using PyTorch and UTKFace Dataset
This notebook demonstrates how to build a simple age, gender, and race prediction model using PyTorch and the UTKFace dataset.
The UTKFace dataset contains face images with annotations for age, gender, and race.
The goal is to create a convolutional neural network (CNN) that can predict these attributes from the images.
The dataset can be downloaded from the following link:
https://susanqq.github.io/UTKFace/

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_DIR = "./UTKFace"  # <-- Ubah ke folder dataset Anda
BATCH_SIZE = 32
NUM_EPOCHS = 150 # Jumlah epoch
LR = 1e-4

In [3]:
class UTKFaceDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.img_names = [img for img in os.listdir(img_dir) if img.endswith('.jpg')]
        self.transform = transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_name = self.img_names[idx]
        path = os.path.join(self.img_dir, img_name)
        image = Image.open(path).convert("RGB")

        # Parse label: [age]_[gender]_[race]_[...] format
        age, gender, race = map(int, img_name.split("_")[:3])

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor([age], dtype=torch.float32), torch.tensor(gender), torch.tensor(race)


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [5]:
dataset = UTKFaceDataset(IMG_DIR, transform=transform)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
class MultiOutputResNet(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(weights='IMAGENET1K_V1')
        self.features = nn.Sequential(*list(base.children())[:-1])
        in_features = base.fc.in_features

        self.age_head = nn.Linear(in_features, 1)
        self.gender_head = nn.Linear(in_features, 2)
        self.race_head = nn.Linear(in_features, 5)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)

        age = self.age_head(x)
        gender = self.gender_head(x)
        race = self.race_head(x)
        return age, gender, race

In [7]:
model = MultiOutputResNet().to(DEVICE)

In [8]:
age_loss_fn = nn.MSELoss()
gender_loss_fn = nn.CrossEntropyLoss()
race_loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [ ]:
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0

    for images, ages, genders, races in train_loader:
        images = images.to(DEVICE)
        ages = ages.to(DEVICE)
        genders = genders.to(DEVICE)
        races = races.to(DEVICE)

        optimizer.zero_grad()

        pred_age, pred_gender, pred_race = model(images)

        loss_age = age_loss_fn(pred_age, ages)
        loss_gender = gender_loss_fn(pred_gender, genders)
        loss_race = race_loss_fn(pred_race, races)

        loss = loss_age + loss_gender + loss_race
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/150], Loss: 452.7658
Epoch [2/150], Loss: 73.2009
Epoch [3/150], Loss: 50.7078
Epoch [4/150], Loss: 38.3509
Epoch [5/150], Loss: 28.3772
Epoch [6/150], Loss: 21.5576
Epoch [7/150], Loss: 17.5735
Epoch [8/150], Loss: 14.0118
Epoch [9/150], Loss: 11.9285
Epoch [10/150], Loss: 10.8906
Epoch [11/150], Loss: 10.4092
Epoch [12/150], Loss: 10.6913
Epoch [13/150], Loss: 9.5668
Epoch [14/150], Loss: 8.5065
Epoch [15/150], Loss: 7.8767
Epoch [16/150], Loss: 7.4071
Epoch [17/150], Loss: 7.1895
Epoch [18/150], Loss: 7.4876
Epoch [19/150], Loss: 6.7455
Epoch [20/150], Loss: 6.2042
Epoch [21/150], Loss: 5.8174
Epoch [22/150], Loss: 5.5818
Epoch [23/150], Loss: 5.6936
Epoch [24/150], Loss: 5.6207
Epoch [25/150], Loss: 5.0096
Epoch [26/150], Loss: 4.9786
Epoch [27/150], Loss: 4.8941
Epoch [28/150], Loss: 4.7088
Epoch [29/150], Loss: 4.3307
Epoch [30/150], Loss: 4.2713
Epoch [31/150], Loss: 4.1801
Epoch [32/150], Loss: 3.8151
Epoch [33/150], Loss: 3.8038
Epoch [34/150], Loss: 3.9816
Epoch [35/

In [10]:
torch.save(model.state_dict(), "multioutput_utkface.pth")
print("✅ Model saved as multioutput_utkface.pth")

✅ Model saved as multioutput_utkface.pth
